In [14]:
from __future__ import print_function

from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
import pickle as pkl
import copy
import scipy
import sparse

In [ ]:
loaddir = '/home/bb596/rds/hpc-work/dl4chem/'
savedir = '/home/bb596/rds/hpc-work/dl4chem/processed/'

def to_onehot(val, cat, etc=0):

    onehot=np.zeros(len(cat))
    for ci, c in enumerate(cat):
        if val == c:
            onehot[ci]=1

    if etc==1 and np.sum(onehot)==0:
        print(val)

    return onehot


def atomFeatures(a, ri_a):

    def _ringSize_a(a, rings):
        onehot = np.zeros(6)
        aid = a.GetIdx()
        for ring in rings:
            if aid in ring and len(ring) <= 8:
                onehot[len(ring) - 3] += 1

        return onehot

    v1 = to_onehot(a.GetSymbol(), ['C','N','O','F','Cl','Br','I','S','B','Si','P','Te','Se','Ge','As'], 1)
    v2 = to_onehot(str(a.GetHybridization()), ['SP','SP2','SP3','SP3D','SP3D2'], 1)

    v3 = [a.GetAtomicNum(), a.GetDegree(), a.GetFormalCharge(), a.GetTotalNumHs(), atom.GetImplicitValence(), a.GetNumRadicalElectrons(), int(a.GetIsAromatic())]
    v4 = _ringSize_a(a, ri_a)

    v5 = np.zeros(3)
    try:
        tmp = to_onehot(a.GetProp('_CIPCode'), ['R','S'], 1)
        v5[0] = tmp[0]
        v5[1] = tmp[1]
    except:
        v5[2]=1

    v5 = v5[:2]

    return np.concatenate([v1,v2,v3,v4,v5], axis=0)


def bondFeatures(bbs, samering, shortpath):

    if len(bbs)==1:
        v1 =  to_onehot(str(bbs[0].GetBondType()), ['SINGLE', 'DOUBLE', 'TRIPLE', 'AROMATIC'], 1)
        v2 = to_onehot(str(bbs[0].GetStereo()), ['STEREOZ', 'STEREOE','STEREOANY','STEREONONE'], 1)
        v2 = v2[:2]
        v3 = [int(bbs[0].GetIsConjugated()), int(bbs[0].IsInRing()), samering, shortpath]
    else:
        v1 = np.zeros(4)
        v2 = np.zeros(2)
        v3 = [0, 0, samering, shortpath]

    return np.concatenate([v1,v2,v3], axis=0)


data='COD'
n_min=2
n_max=50
atom_dim=35
edge_dim=10

[mollist, smilist] = pkl.load(open(loaddir+data+'_molset_all.p','rb'))

D1 = []
D2 = []
D3 = []
D4 = []
D5 = []
mollist2 = []
smilist2 = []
print(len(mollist), flush=True)
for i in range(len(mollist)):
    if i % 1000 == 0: print(i, flush=True)

    smi = smilist[i]
    mol = mollist[i]

    Chem.rdmolops.AssignAtomChiralTagsFromStructure(mol)
    Chem.rdmolops.AssignStereochemistry(mol)

    if mol.GetNumHeavyAtoms() < n_min or mol.GetNumHeavyAtoms() > n_max:
        print('error')
        break

    n = mol.GetNumAtoms()
    ri = mol.GetRingInfo()
    ri_a = ri.AtomRings()

    pos = mol.GetConformer().GetPositions()

    assert n==pos.shape[0]

    mollist2.append(mol)
    smilist2.append(smi)

    node = np.zeros((n_max, atom_dim))
    mask = np.zeros((n_max, 1))

    for j in range(n):
        atom = mol.GetAtomWithIdx(j)
        node[j, :]=atomFeatures(atom, ri_a)
        mask[j, 0]=1

    edge = np.zeros((n_max, n_max, edge_dim))

    for j in range(n-1):
        for k in range(j+1, n):
            molpath = Chem.GetShortestPath(mol, j, k)
            shortpath = len(molpath) - 1
            assert shortpath>0

            samering = 0
            for alist in ri_a:
                if j in alist and k in alist:
                    samering = 1

            bond = [mol.GetBondBetweenAtoms(molpath[mm], molpath[mm+1]) for mm in range(shortpath)]

            edge[j, k, :] = bondFeatures(bond, samering, shortpath)
            edge[k, j, :] = edge[j, k, :]

    proximity = np.zeros((n_max, n_max))
    proximity[:n, :n] = euclidean_distances(pos)

    pos2 = np.zeros((n_max, 3))
    pos2[:n] = pos

    D1.append(np.array(node, dtype=int))
    D2.append(np.array(mask, dtype=int))
    D3.append(np.array(edge, dtype=int))
    D4.append(np.array(proximity))
    D5.append(np.array(pos2))

    #if len(D1)==66000:
    #    break

D1 = np.array(D1, dtype=int)
D2 = np.array(D2, dtype=int)
D3 = np.array(D3, dtype=int)
D4 = np.array(D4)
D5 = np.array(D5)

print([D1.shape, D2.shape, D3.shape, D4.shape, D5.shape])
print([np.sum(np.isnan(D1)), np.sum(np.isnan(D2)), np.sum(np.isnan(D3)), np.sum(np.isnan(D4))])
print([D1.nbytes, D3.nbytes]) # [933282000, 13332600000]

In [15]:
D1 = sparse.COO.from_numpy(D1)
D2 = sparse.COO.from_numpy(D2)
D3 = sparse.COO.from_numpy(D3)
print([D1.nbytes, D3.nbytes])

[329165824, 2273547040]


In [17]:
molvec_fname = savedir + data+'_molvec_'+str(n_max)+'.p'
molset_fname = savedir + data + '_molset_' + str(n_max) + '.p'

print(molvec_fname)
print(molset_fname)

with open(molvec_fname,'wb') as f:
    pkl.dump([D1, D2, D3, D4, D5], f)

mollist2 = np.array(mollist2)
smilist2 = np.array(smilist2)

with open(molset_fname,'wb') as f:
    pkl.dump([mollist2, smilist2], f)

/home/bb596/rds/hpc-work/dl4chem/processed/COD_molvec_50.p
/home/bb596/rds/hpc-work/dl4chem/processed/COD_molset_50.p
